# PyRIT Initialization System

This notebook explains the PyRIT initialization system, including the `initialize_pyrit` function and the `PyRITInitializer` class system for structured configuration.

## Overview

PyRIT provides two complementary ways to configure default values and global settings:

1. **Built-in PyRITInitializer classes** - Stable, tested, documented configuration classes included with PyRIT
2. **External PyRITInitializer scripts** - Custom PyRITInitializer classes in external files for flexible, project-specific configuration

Both approaches use the same PyRITInitializer base class, ensuring type safety and consistent behavior. External scripts are particularly powerful for rapid prototyping, custom configurations, and project-specific needs.

## Execution Order

When `initialize_pyrit` is called, the following happens in order:

1. **Environment files** are loaded (`.env`, `.env.local`)
2. **Default values are reset** to ensure clean state
3. **Memory database** is configured (SQLite, AzureSQL, or InMemory)
4. **All PyRITInitializer instances are collected** - both direct instances and those loaded from external scripts
5. **All initializers are sorted by `execution_order`** property and executed in sequence

This unified approach means all configuration goes through the same PyRITInitializer system, whether built-in or external.

## PyRITInitializer Classes

PyRITInitializer classes provide a structured, type-safe way to configure PyRIT. All configuration in PyRIT now goes through this class system, whether built-in or external.

### Key Benefits

- **Type Safety**: Full type checking and IDE support for all configurations
- **Validation**: Built-in validation before execution
- **Discoverability**: Easy to find and understand what they configure
- **Documentation**: Self-documenting with clear descriptions
- **Execution Order**: Fine-grained control over initialization sequence across all initializers
- **Dynamic Info**: Automatic tracking of what defaults are actually set
- **Flexibility**: Can be built-in (stable) or external (customizable)

### Built-in vs External Initializers

**Built-in Initializers:**
- Stable, well-tested configurations included with PyRIT
- Examples: SimpleInitializer, AIRTInitializer
- Good for standard use cases

**External Initializers:**
- Custom PyRITInitializer classes in separate `.py` files
- Perfect for project-specific needs, rapid prototyping
- Can be easily shared across projects or later integrated into PyRIT
- Same type safety and validation as built-in classes

### Creating a Custom Initializer

Here's how to create your own PyRITInitializer (works the same for built-in or external):

In [ ]:
from pyrit.setup.initializers.base import PyRITInitializer
from pyrit.common.apply_defaults import set_default_value
from pyrit.prompt_target import OpenAIChatTarget
import os

class CustomInitializer(PyRITInitializer):
    """Custom configuration for my specific use case."""
    
    @property
    def name(self) -> str:
        return "Custom Configuration"
    
    @property
    def description(self) -> str:
        return "Sets up custom OpenAI defaults with high temperature for creative tasks"
    
    @property
    def execution_order(self) -> int:
        return 2  # Run after basic setup (default is 1)
    
    def validate(self) -> None:
        """Custom validation logic."""
        if not os.getenv("OPENAI_API_KEY"):
            raise ValueError("OPENAI_API_KEY environment variable is required")
    
    def initialize(self) -> None:
        """Configure default values."""
        # Set high temperature for creative tasks
        set_default_value(
            class_type=OpenAIChatTarget,
            parameter_name="temperature",
            value=0.9
        )
        
        # Set default model
        set_default_value(
            class_type=OpenAIChatTarget,
            parameter_name="deployment_name", 
            value="gpt-4"
        )

# Create an instance
custom_init = CustomInitializer()

# View what it configures
print("Custom Initializer Info:")
for key, value in custom_init.get_info().items():
    print(f"  {key}: {value}")

### Built-in Initializers

PyRIT includes several built-in initializers accessible through `InitializationPaths`:

In [1]:
from pyrit.setup import InitializationPaths

# Get built-in initializers
simple_init = InitializationPaths.get_simple_initializer()
airt_init = InitializationPaths.get_airt_initializer()

print("Simple Initializer:")
for key, value in simple_init.get_info().items():
    print(f"  {key}: {value}")

print("\nAIRT Initializer:")
for key, value in airt_init.get_info().items():
    print(f"  {key}: {value}")

Simple Initializer:
  name: Simple Complete Configuration
  description: Complete simple setup with basic OpenAI converters, objective scorer (no harm detection), and adversarial targets. Only requires OPENAI_API_KEY environment variable.
  class: SimpleInitializer
  execution_order: 1
  default_values: Preview unavailable - PyRIT memory system not accessible
  global_variables: Preview unavailable - PyRIT memory system not accessible
  required_env_vars: ['OPENAI_CHAT_ENDPOINT', 'OPENAI_CHAT_KEY']
  profile: simple

AIRT Initializer:
  name: AIRT Default Configuration
  description: AI Red Team setup with Azure OpenAI converters, composite harm/objective scorers, and adversarial targets
  class: AIRTInitializer
  execution_order: 1
  default_values: Preview unavailable - PyRIT memory system not accessible
  global_variables: Preview unavailable - PyRIT memory system not accessible
  required_env_vars: ['AZURE_OPENAI_GPT4O_UNSAFE_ENDPOINT', 'AZURE_OPENAI_GPT4O_UNSAFE_ENDPOINT_KEY', 'AZ

### Execution Order Control

The `execution_order` property controls when initializers run relative to each other:

In [ ]:
class EarlySetupInitializer(PyRITInitializer):
    @property
    def name(self) -> str:
        return "Early Setup"
    
    @property
    def description(self) -> str:
        return "Basic configuration that should run first"
        
    @property
    def execution_order(self) -> int:
        return 0  # Runs before default (1)
        
    def initialize(self) -> None:
        # Early setup logic here
        pass

class LateOverrideInitializer(PyRITInitializer):
    @property
    def name(self) -> str:
        return "Late Override"
        
    @property
    def description(self) -> str:
        return "Final configuration overrides"
        
    @property
    def execution_order(self) -> int:
        return 10  # Runs after most others
        
    def initialize(self) -> None:
        # Override logic here
        pass

# When passed to initialize_pyrit, these will execute in order:
# 1. EarlySetupInitializer (order=0)
# 2. Standard initializers (order=1, default)
# 3. LateOverrideInitializer (order=10)

early = EarlySetupInitializer()
late = LateOverrideInitializer()

print(f"Early setup execution order: {early.execution_order}")
print(f"Late override execution order: {late.execution_order}")

## External PyRITInitializer Scripts

External initialization scripts are Python files that contain PyRITInitializer classes. They provide powerful flexibility for:

- **Custom configurations** for specific projects or use cases
- **Rapid prototyping** of new configurations before integrating them
- **Project-specific needs** when you have PyRIT checked out
- **Sharing configurations** across different projects without modifying core PyRIT
- **Custom targets** or specialized setups for specific workflows

### Script Requirements

External scripts must:
- Be valid Python files (`.py` extension) 
- Contain one or more PyRITInitializer classes
- Export initializers via either:
  - A `get_initializers()` function that returns a list of PyRITInitializer instances
  - An `initializers` variable containing a list of PyRITInitializer instances

### Example External Script

Here's an example of what an external initializer script looks like:

In [ ]:
import tempfile
import os

# Create a temporary directory for our example
temp_dir = tempfile.mkdtemp()
script_path = os.path.join(temp_dir, "external_custom_initializers.py")

# Example content for external_custom_initializers.py
script_content = '''
"""External PyRIT initializers for custom project configuration."""
import os
from pyrit.setup.initializers.base import PyRITInitializer
from pyrit.common.apply_defaults import set_default_value, set_global_variable
from pyrit.prompt_target import OpenAIChatTarget


class ProjectSpecificInitializer(PyRITInitializer):
    """Custom initializer for my specific project needs."""
    
    @property
    def name(self) -> str:
        return "Project-Specific Configuration"
    
    @property
    def description(self) -> str:
        return "Custom OpenAI configuration optimized for creative writing tasks"
    
    @property
    def execution_order(self) -> int:
        return 5  # Run after most built-in initializers
    
    def validate(self) -> None:
        """Validate project-specific requirements."""
        if not os.getenv("MY_PROJECT_ENDPOINT"):
            raise ValueError("MY_PROJECT_ENDPOINT environment variable is required")
    
    def initialize(self) -> None:
        """Configure project-specific defaults."""
        # Set up custom endpoint
        custom_endpoint = os.getenv("MY_PROJECT_ENDPOINT")
        set_global_variable(name="PROJECT_ENDPOINT", value=custom_endpoint)
        
        # High temperature for creative tasks
        set_default_value(
            class_type=OpenAIChatTarget,
            parameter_name="temperature",
            value=0.95
        )
        
        # Custom token limit for longer responses
        set_default_value(
            class_type=OpenAIChatTarget,
            parameter_name="max_tokens",
            value=4000
        )


class ExperimentalInitializer(PyRITInitializer):
    """Experimental configuration I'm testing."""
    
    @property
    def name(self) -> str:
        return "Experimental Configuration"
    
    @property
    def description(self) -> str:
        return "Testing new configuration patterns"
    
    @property
    def execution_order(self) -> int:
        return 0  # Run very early
    
    def initialize(self) -> None:
        """Experimental setup logic."""
        # This might become a built-in initializer later if it works well
        pass

'''

# Write the example script to temporary directory
with open(script_path, "w") as f:
    f.write(script_content)
    
print(f"Example external script created in temp directory: {script_path}")
print("\nThis script contains PyRITInitializer classes that can be loaded by initialize_pyrit")
print("Key features:")
print("  • Type-safe PyRITInitializer classes")
print("  • Custom validation logic")
print("  • Execution order control")
print("  • Project-specific configuration")
print("  • Same API as built-in initializers")


## Using initialize_pyrit

The `initialize_pyrit` function accepts both initializer classes and script paths:

In [ ]:
from pyrit.setup import initialize_pyrit, InitializationPaths

# Example 1: Using only class-based initializers (recommended)
initialize_pyrit(
    memory_db_type="InMemory",
    initializers=[InitializationPaths.get_simple_initializer()]
)

print("✅ Initialized with class-based initializer")

In [ ]:
# Example 2: Using external PyRITInitializer scripts
initialize_pyrit(
    memory_db_type="InMemory",
    initialization_scripts=[script_path]  # Use the temp file path
)

print("✅ Initialized with external PyRITInitializer script")
print("External scripts must contain PyRITInitializer classes for type safety")
print(f"Global variable PROJECT_ENDPOINT: {globals().get('PROJECT_ENDPOINT', 'Not set')}")


In [ ]:
# Example 3: Using both built-in and external initializers together
initialize_pyrit(
    memory_db_type="InMemory",
    initializers=[
        InitializationPaths.get_simple_initializer(),  # Built-in foundation
        custom_init  # Our custom initializer from earlier
    ],
    initialization_scripts=[script_path]  # External customizations from temp directory
)

print("✅ Initialized with both built-in and external initializers")
print("All initializers are sorted by execution_order and executed in sequence:")
print("  1. ExperimentalInitializer (execution_order=0)")
print("  2. SimpleInitializer (execution_order=1, default)")
print("  3. CustomInitializer (execution_order=2)")
print("  4. ProjectSpecificInitializer (execution_order=5)")
print("\nAll use the same PyRITInitializer API for consistency and type safety")


## Developing Custom Configurations

The external script approach is particularly powerful for development workflows:

### Rapid Prototyping
```python
# In my_experiment.py - quickly test new configurations
class ExperimentalInitializer(PyRITInitializer):
    def initialize(self) -> None:
        # Try new default values
        set_default_value(OpenAIChatTarget, "temperature", 0.95)

def get_initializers():
    return [ExperimentalInitializer()]

# Test it immediately:
initialize_pyrit(
    memory_db_type="InMemory",
    initialization_scripts=["my_experiment.py"]
)
```

### Project-Specific Configurations
```python
# In project_config.py - custom config for this project
class MyProjectInitializer(PyRITInitializer):
    def initialize(self) -> None:
        # Project-specific target with custom endpoint
        custom_target = OpenAIChatTarget(endpoint=os.getenv("PROJECT_ENDPOINT"))
        set_default_value(PromptConverter, "converter_target", custom_target)
```

### Integration Path
External initializers can easily become built-in initializers:
1. **Develop externally** - iterate quickly in external scripts
2. **Test thoroughly** - validate the configuration works well
3. **Move to PyRIT** - copy the class into pyrit/setup/initializers/
4. **Update InitializationPaths** - add factory method for the new initializer

## Best Practices

1. **Use external scripts for customization** - they're designed for flexibility
2. **Start with built-in initializers** - they provide good foundations
3. **Use execution_order** to control when your customizations apply
4. **Validate early** in the `validate()` method
5. **Document your external initializers** - they can be shared across projects
6. **Consider contributing** useful external initializers back to PyRIT

## Clean up

Let's clean up the example script we created:

In [ ]:
import shutil

# Clean up the temporary directory and all its contents
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
    print(f"Cleaned up temporary directory: {temp_dir}")
else:
    print("Temporary directory already cleaned up")

print("\n🎉 Summary:")
print("PyRIT's initialization system provides:")
print("  • Built-in initializers for standard configurations")
print("  • External scripts for custom, project-specific needs")
print("  • Unified PyRITInitializer API for type safety")
print("  • Execution order control across all initializers")
print("  • Easy path from external prototypes to built-in initializers")


## Choosing Your Initialization Approach

PyRIT provides flexibility in how you configure defaults through the initialization system:

### Built-in Initializers (Recommended for Standard Use Cases)

- **SimpleInitializer**: Basic OpenAI configuration, minimal setup required
- **AIRTInitializer**: Full AIRT configuration using Azure OpenAI setup
- Well-tested, documented, and maintained by the PyRIT team
- Good starting point for most users
- Stable and unlikely to change between PyRIT versions

### External Scripts (Recommended for Custom/Project-Specific Needs)

- Create PyRITInitializer classes in external `.py` files
- Perfect for rapid prototyping and experimentation
- Ideal when you have PyRIT checked out and need custom configurations
- Great for sharing project-specific configurations across teams
- Can be easily moved into the main library later if proven useful
- Full type safety and validation just like built-in initializers

### Hybrid Approach (Best for Extending Standard Configurations)

Combine both approaches - use built-in initializers as a foundation, then add external scripts for customization:

```python
from pyrit.setup import initialize_pyrit
from pyrit.setup.initializers import SimpleInitializer

initialize_pyrit(
    memory_db_type="InMemory",
    initializers=[SimpleInitializer()],            # Foundation
    initialization_scripts=["my_project_config.py"]  # Project-specific tweaks
)
```

This hybrid approach gives you:
- **Stable foundation** from well-tested built-in initializers
- **Customization flexibility** for project-specific needs
- **Easy upgrades** - built-in initializers stay current with PyRIT updates
- **Local control** - external scripts can be version controlled with your project

### Choosing the Right Approach

**Use built-in initializers when:**
- You need standard OpenAI or Azure OpenAI configuration
- You want PyRIT to handle configuration updates automatically
- You're just getting started with PyRIT
- You don't have special requirements

**Use external scripts when:**
- You need custom target configurations for specific projects
- You're prototyping new configuration patterns
- You have unique environment or deployment requirements
- You want to share configurations across different projects
- You're developing new initializers that might become built-in later

**Use hybrid approach when:**
- You want a stable foundation with custom enhancements
- You need standard configs plus project-specific defaults
- You want to gradually transition from external to built-in configs

For detailed information on how default values work with the initialization system, see the [Default Values](default_values.ipynb) notebook.
